<a href="https://colab.research.google.com/github/timkabot/NeuralNetworkMusicGenerator/blob/master/GANS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  print("Tensorflow version is", tf.__version__)

Tensorflow version is 1.15.0


In [ ]:
class CGAN():
  def __init__(self, embeddings, grouped_embeddings, sample_interval, plot_interval):
    self.generator_losses = []
    self.discriminator_losses = []
    self.loss_saving_rate = 1
    self.loss_plot_rate = plot_interval
    self.distribution_plot_rate = sample_interval

    self.all_normalized_lyrics_embeddings = embeddings
    self.grouped_embeddings = grouped_embeddings

    self.song_size = 20
    self.noise_size = 30
    self.label_size = 20
    self.optimizer = Adam(lr=0.0002, beta_1=0.5)

    self.discriminator = self.build_discriminator()
    self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=self.optimizer,
            metrics=['accuracy'])
    self.generator = self.build_generator()

    noise = Input(shape = (self.song_size,  self.noise_size), name="noise")
    labels = Input(shape = (self.song_size, self.label_size), name="labels")
    generated_midi_sequence = self.generator([noise, labels])

    # For the combined model we will only train the generator
    self.discriminator.trainable = False
    valid = self.discriminator([generated_midi_sequence, labels])

    # The combined model  (stacked generator and discriminator)
    # Trains generator to fool discriminator
    self.combined = keras.Model([noise,labels], valid, name='Combined')
    self.combined.compile(loss=['binary_crossentropy'],  # BCE(D(G(Z)),1))
        optimizer=self.optimizer)

  def build_generator(self):
    model = keras.Sequential( [
    Dense(400, input_shape=(self.song_size, self.noise_size + 20)),
    LeakyReLU(alpha=0.2),
    LSTM(400, return_sequences=True, activation = "tanh"),    
    Dropout(0.25),
    LSTM(400, return_sequences=True, activation = "tanh"),
    Dense(3, activation = "tanh")
    ])

    model.build((self.song_size, self.noise_size + 20))
    model.summary()
    
    noise  = Input(shape=(self.song_size,  self.noise_size), name='z_input')
    labels = Input(shape=(self.song_size, self.label_size), name='class_labels')
    
    generator_input = Concatenate()([noise, labels])
    outputs = model(generator_input)
  
    return keras.Model([noise, labels], outputs, name='generator')

  def build_discriminator(self):
    model = keras.Sequential([
    LSTM(400, return_sequences=True, input_shape=(self.song_size, self.label_size + 3), activation = "tanh"),
    Dropout(0.25),
    LSTM(400, activation="tanh"),
    Flatten(),
    Dense(1, activation='sigmoid')])
        
    model.build((self.song_size, self.label_size + 3))
    model.summary()

    notes  = Input(shape=(self.song_size, 3), name="notes")
    labels = Input(shape=(self.song_size, self.label_size), name="labels")
    model_input = Concatenate()([notes, labels])

    validity = model(model_input)
    
    return keras.Model([notes,labels], outputs=validity, name='discriminator')

  def save_models(self, epoch):
    self.generator.save_weights("/content/drive/My Drive/Thesis/SecondSemester/CGAN/generator_epoch_%d.h5" % epoch)
    self.discriminator.save_weights("/content/drive/My Drive/Thesis/SecondSemester/CGAN/discriminator_epoch_%d.h5" % epoch)
    self.combined.save_weights("/content/drive/My Drive/Thesis/SecondSemester/CGAN/combined_epoch_%d.h5" % epoch)

  def train(self, epochs, X_train, y_train,song_size, batch_size=1):
    valid = np.ones((batch_size, 1))  # Adversarial ground truths
    fake = np.zeros((batch_size, 1))  # Fake labels
    for epoch in tqdm_notebook(range(1, epochs)):
      # Train DISCRIMINATOR
      song_number = np.random.randint(0, len(X_train), batch_size)
      notes, labels = X_train[song_number], y_train[song_number]

      notes = notes.reshape(batch_size, song_size, 3)
      labels = labels.reshape(batch_size, song_size , self.label_size)

      noise = generate_latent_points(self.noise_size, song_size, batch_size) # generating noise
      gen_notes = self.generator.predict([noise, labels])

      if(epoch % self.loss_saving_rate == 0):
        d_loss_real = self.discriminator.train_on_batch([notes, labels], valid)
        d_loss_fake = self.discriminator.train_on_batch([gen_notes, labels], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) # 0th for real, 1st for fake
        self.discriminator_losses.append((epoch, (d_loss[0], d_loss[1]))) # add discriminator loss

      # Train GENERATOR
      
      # Condition on labels
      fake_labels = self.all_normalized_lyrics_embeddings[np.random.randint(0, len(self.all_normalized_lyrics_embeddings), self.song_size * batch_size)].reshape(batch_size, self.song_size, self.label_size) 
      g_loss = self.combined.train_on_batch([noise, fake_labels], valid) # BCE(D(G(Z)),1))
      if(epoch % self.loss_saving_rate == 0):
        self.generator_losses.append((epoch, g_loss))  # add generator loss
      
      # Plot loss
      if(epoch % self.loss_plot_rate == 0):
        plot_loss(self.generator_losses, self.discriminator_losses)
        
      #plot distribution
      if(epoch % self.distribution_plot_rate == 0):
        plot_notes_distribution(epoch, self.generator, self.noise_size, self.grouped_embeddings)
        self.save_models(epoch)


In [ ]:
class CGAN_anylength():
  def __init__(self, embeddings, grouped_embeddings, sample_interval, plot_interval):
    self.generator_losses = []
    self.discriminator_losses = []
    self.loss_saving_rate = 1
    self.loss_plot_rate = plot_interval
    self.distribution_plot_rate = sample_interval

    self.all_normalized_lyrics_embeddings = embeddings
    self.grouped_embeddings = grouped_embeddings

    self.song_size = 20
    self.noise_size = 30
    self.label_size = 20
    self.optimizer = Adam(lr=0.0002, beta_1=0.5)

    self.discriminator = self.build_discriminator()
    self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=self.optimizer,
            metrics=['accuracy'])
    self.generator = self.build_generator()

    noise = Input(shape = (None,  self.noise_size), name="noise")
    labels = Input(shape = (None, self.label_size), name="labels")
    generated_midi_sequence = self.generator([noise, labels])

    # For the combined model we will only train the generator
    self.discriminator.trainable = False
    valid = self.discriminator([generated_midi_sequence, labels])

    # The combined model  (stacked generator and discriminator)
    # Trains generator to fool discriminator
    self.combined = keras.Model([noise,labels], valid, name='Combined')
    self.combined.compile(loss=['binary_crossentropy'],  # BCE(D(G(Z)),1))
        optimizer=self.optimizer)

  def build_generator(self):
    model = keras.Sequential( [
    Dense(400, input_shape=(None, self.noise_size + 20)),
    LeakyReLU(alpha=0.2),
    LSTM(400, return_sequences=True, activation = "tanh"),    
    Dropout(0.25),
    LSTM(400, return_sequences=True, activation = "tanh"),
    Dense(3, activation = "tanh")
    ])

    model.build((None, self.noise_size + 20))
    model.summary()
    
    noise  = Input(shape=(self.song_size,  self.noise_size), name='z_input')
    labels = Input(shape=(self.song_size, self.label_size), name='class_labels')
    
    generator_input = Concatenate()([noise, labels])
    outputs = model(generator_input)
  
    return keras.Model([noise, labels], outputs, name='generator')

  def build_discriminator(self):
    model = keras.Sequential([
    LSTM(400, return_sequences=True, 
         input_shape=(None, self.label_size + 3), activation = "tanh"),
    Dropout(0.25),
    LSTM(400, activation="tanh"),
    Flatten(),
    Dense(1, activation='sigmoid')])
        
    model.build((None, self.label_size + 3))
    model.summary()

    notes  = Input(shape=(self.song_size, 3), name="notes")
    labels = Input(shape=(self.song_size, self.label_size), name="labels")
    model_input = Concatenate()([notes, labels])

    validity = model(model_input)
    
    return keras.Model([notes,labels], outputs=validity, name='discriminator')

  def save_models(self, epoch):
    self.generator.save_weights("/content/drive/My Drive/Thesis/SecondSemester/CGAN_anylength/generator_epoch_%d.h5" % epoch)
    self.discriminator.save_weights("/content/drive/My Drive/Thesis/SecondSemester/CGAN_anylength/discriminator_epoch_%d.h5" % epoch)
    self.combined.save_weights("/content/drive/My Drive/Thesis/SecondSemester/CGAN_anylength/combined_epoch_%d.h5" % epoch)

  def train(self, epochs, X_train, y_train,song_size, batch_size=1):
    valid = np.ones((batch_size, 1))  # Adversarial ground truths
    fake = np.zeros((batch_size, 1))  # Fake labels
    for epoch in tqdm_notebook(range(1, epochs)):
      # Train DISCRIMINATOR
      song_number = np.random.randint(0, len(X_train), batch_size)
      notes, labels = X_train[song_number], y_train[song_number]

      notes = notes.reshape(batch_size, song_size, 3)
      labels = labels.reshape(batch_size, song_size , self.label_size)

      noise = generate_latent_points(self.noise_size, song_size, batch_size) # generating noise
      gen_notes = self.generator.predict([noise, labels])

      if(epoch % self.loss_saving_rate == 0):
        d_loss_real = self.discriminator.train_on_batch([notes, labels], valid)
        d_loss_fake = self.discriminator.train_on_batch([gen_notes, labels], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) # 0th for real, 1st for fake
        self.discriminator_losses.append((epoch, (d_loss[0], d_loss[1]))) # add discriminator loss

      # Train GENERATOR
      
      # Condition on labels
      fake_labels = self.all_normalized_lyrics_embeddings[np.random.randint(0, len(self.all_normalized_lyrics_embeddings), self.song_size * batch_size)].reshape(batch_size, self.song_size, self.label_size) 
      g_loss = self.combined.train_on_batch([noise, fake_labels], valid) # BCE(D(G(Z)),1))
      if(epoch % self.loss_saving_rate == 0):
        self.generator_losses.append((epoch, g_loss))  # add generator loss
      
      # Plot loss
      if(epoch % self.loss_plot_rate == 0):
        plot_loss(self.generator_losses, self.discriminator_losses)
        
      #plot distribution
      if(epoch % self.distribution_plot_rate == 0):
        plot_notes_distribution(epoch, self.generator, self.noise_size, self.grouped_embeddings)
        self.save_models(epoch)


In [ ]:
class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

class CWGAN_GP():
  def __init__(self, embeddings, grouped_embeddings, sample_interval, plot_interval):
    self.generator_losses = []
    self.discriminator_losses = []
    self.loss_saving_rate = 5
    self.loss_plot_rate = plot_interval
    self.distribution_plot_rate = sample_interval
    self.all_normalized_lyrics_embeddings = embeddings
    self.grouped_embeddings = grouped_embeddings

    self.song_size = 20
    self.noise_size = 100
    self.label_size = 20
    self.optimizer = RMSprop(lr=0.00005)
    self.n_critic = 5
    
    # Build the generator and critic
    self.generator = self.build_generator()
    self.critic = self.build_critic()

    #-------------------------------
    # Construct Computational Graph
    #       for the Critic
    #-------------------------------

    # Freeze generator's layers while training critic
    self.generator.trainable = False

    # Song input (real sample)
    real_song = Input(shape=(self.song_size,  3))

    # Noise input
    noise = Input(shape=(self.song_size,  self.noise_size))
    # Generate sequence based of noise (fake sample) and add labels to the input 
    labels = Input(shape = (self.song_size, self.label_size), name="labels")
    fake_song = self.generator([noise, labels])

    # Discriminator determines validity of the real and fake images
    fake = self.critic([fake_song, labels])
    valid = self.critic([real_song, labels])

    # Construct weighted average between real and fake sequences
    interpolated_song = RandomWeightedAverage()([real_song, fake_song])
    
    # Determine validity of weighted sample
    validity_interpolated = self.critic([interpolated_song, labels])

    # Use Python partial to provide loss function with additional
    # 'averaged_samples' argument
    partial_gp_loss = partial(self.gradient_penalty_loss,
                      averaged_samples=interpolated_song)
    partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

    self.critic_model = Model(inputs=[real_song, labels, noise], outputs=[valid, fake, validity_interpolated])
    self.critic_model.compile(loss=[self.wasserstein_loss,
                                    self.wasserstein_loss,
                                    partial_gp_loss],
                                    optimizer=self.optimizer,
                                    loss_weights=[1, 1, 10])
    #-------------------------------
    # Construct Computational Graph
    #         for Generator
    #-------------------------------

    # For the generator we freeze the critic's layers
    self.critic.trainable = False
    self.generator.trainable = True

    # Sampled noise for input to generator
    noise = Input(shape=(self.song_size,  self.noise_size))
    # add label to the input
    labels = Input(shape = (self.song_size, self.label_size), name="labels")
    # Generate song based of noise
    fake_song = self.generator([noise, labels])
    # Discriminator determines validity
    valid = self.critic([fake_song, labels])
    # Defines generator model
    self.generator_model = Model([noise, labels], valid)
    self.generator_model.compile(loss=self.wasserstein_loss, optimizer=self.optimizer)

  def build_generator(self):
    model = keras.Sequential([
    Dense(400, input_shape=(self.song_size, self.noise_size + 20)),
    LeakyReLU(alpha=0.2),
    LSTM(400, return_sequences=True, activation = "tanh",  unroll=True),    
    LSTM(400, return_sequences=True, activation = "tanh",  unroll=True),
    Dense(3, activation = "tanh")
    ])
    
    noise  = Input(shape=(self.song_size,  self.noise_size), name='z_input')
    labels = Input(shape=(self.song_size, self.label_size), name='class_labels')
    generator_input = Concatenate()([noise, labels])
    outputs = model(generator_input)
  
    return keras.Model([noise, labels], outputs, name='generator')

  def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
    """
    Computes gradient penalty based on prediction and weighted real / fake samples
    """
    gradients = K.gradients(y_pred, averaged_samples)[0]
    # compute the euclidean norm by squaring ...
    gradients_sqr = K.square(gradients)
    #   ... summing over the rows ...
    gradients_sqr_sum = K.sum(gradients_sqr,
                              axis=np.arange(1, len(gradients_sqr.shape)))
    #   ... and sqrt
    gradient_l2_norm = K.sqrt(gradients_sqr_sum)
    # compute lambda * (1 - ||grad||)^2 still for each single sample
    gradient_penalty = K.square(1 - gradient_l2_norm)
    # return the mean as loss over all the batch samples
    return K.mean(gradient_penalty)


  def wasserstein_loss(self, y_true, y_pred):
    return K.mean(y_true * y_pred)

  def save_models(self, epoch):
    self.generator.save_weights("/content/drive/My Drive/Thesis/SecondSemester/CWGAN_GP/generator_epoch_%d.h5" % epoch)
    self.critic.save_weights("/content/drive/My Drive/Thesis/SecondSemester/CWGAN_GP/critic_epoch_%d.h5" % epoch)
    self.critic_model.save_weights("/content/drive/My Drive/Thesis/SecondSemester/CWGAN_GP/critic_model_epoch_%d.h5" % epoch)
    self.generator_model.save_weights("/content/drive/My Drive/Thesis/SecondSemester/CWGAN_GP/generator_model_epoch_%d.h5" % epoch)
  def build_critic(self):
    model = keras.Sequential([
    LSTM(400, return_sequences=True, input_shape=(self.song_size, self.label_size + 3), activation = "tanh", unroll=True),
    LSTM(400, activation="tanh",  unroll=True),
    Dense(1)])

    notes  = Input(shape=(self.song_size, 3))
    labels = Input(shape=(self.song_size, self.label_size))
    model_input = Concatenate()([notes, labels])
    validity = model(model_input)
    
    return keras.Model([notes,labels], outputs=validity, name='discriminator')

  def train(self, epochs, X_train, y_train, song_size, batch_size=16):
    valid = -np.ones((batch_size, 1))
    fake = np.ones((batch_size, 1))
    dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty

    for epoch in tqdm_notebook(range(1, epochs)):
      for _ in range(self.n_critic):
         # ---------------------
         #  Train Critic
         # ---------------------
         idx = np.random.randint(0, len(X_train), batch_size)

         notes = X_train[idx].reshape(batch_size, song_size, 3)
         labels = y_train[idx].reshape(batch_size, song_size , self.label_size)
         
         noise = generate_latent_points(self.noise_size, song_size, batch_size) # generating noise
         d_loss = self.critic_model.train_on_batch([notes, labels, noise], [valid, fake, dummy])

        
     
      # ---------------------
      #  Train Generator
      # ---------------------
     
      noise = generate_latent_points(self.noise_size, song_size, batch_size) # generating noise
      fake_labels = self.all_normalized_lyrics_embeddings[np.random.randint(0, len(self.all_normalized_lyrics_embeddings), self.song_size * batch_size)].reshape(batch_size, self.song_size, self.label_size) 
      g_loss = self.generator_model.train_on_batch([noise, fake_labels], valid)

      if(epoch % self.loss_saving_rate == 0):
        self.generator_losses.append((epoch, g_loss))  # add generator loss
        self.discriminator_losses.append((epoch, (d_loss[0], d_loss[1]))) # add discriminator loss
      # Plot distribution
      if(epoch % self.distribution_plot_rate == 0):
        plot_notes_distribution(epoch, self.generator, self.noise_size, self.grouped_embeddings)
        self.save_models(epoch)
      # Plot loss
      if(epoch % self.loss_plot_rate == 0):
        plot_loss(self.generator_losses, self.discriminator_losses)

NameError: ignored